# Лабораторная работа №2 по курсу “Компьютерная лингвистика”

#### Выполнил: студент группы 381703-4м Четвериков Антон

## Постановка задачи


В первой части нужно создать переводчик с украинского языка на русский на основе домашней работы #1 с Яндекс ШАД.

Вторая часть лабораторной работы состоит в исследовании свойств кросс-языкового отображения.
Предлагается использовать модель FastText для векторных репрезентаций слов. Для модели FastText можно найти и использовать предобученные векторы.

Необходимо загрузить две модели векторных репрезентаций (по аналогии с домашней работой из ШАД), и построить матрицу отображения между двумя пространствами. 

В качестве слов соответствия можно взять словари, поставляемые FaceBook (Ground-truth bilingual dictionaries).

Взять 2 набора по 300-500 слов из словаря отображений слов (обозначим их pair1 и pair2) и построить две матрицы M1, M2 отображений между пространствами векторных репрезентаций. Для решения матричного уравнения воспользуйтесь функцией np.linalg.lstsq (в NumPy). 

* Подсчитайте точность перевода языковых пар pair1, pair2 для матрицы M1
* Подсчитайте точность перевода языковых пар pair1, pair2 для матрицы M2
* Подсчитайте норму Фробениуса для матрицы M1-M2
* Подсчитайте: |M1-M2|/|M1| (норма - Фробениусова).
* Обладает ли матрица отображения свойством линейности?
* Ортогональны ли матрицы M1 и M2?

Повторите процедуру в обе стороны: французский -> португальский и португальский -> французский

## Часть 1: переводчик с украинского на русский

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings('ignore')

Загружаем векторные представления слов для русского и украинского языков:

In [2]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec", limit = 100000)

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec", limit = 100000)

Загружаем небольшие словари для соответствующих пар слов в качестве тренировочного и тестового множеств:

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028262138367),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087710380554),
 ('декабрь', 0.8652558326721191),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096439480781555),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992520570755005),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592805862426758),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256430625915527)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('одностороннего', 0.22608889639377594),
 ('подход', 0.22305874526500702),
 ('конструктивное', 0.21656130254268646),
 ('подхода', 0.2142382711172104),
 ('аспектах', 0.21134456992149353),
 ('двустороннего', 0.21055522561073303),
 ('Продление', 0.2092539370059967),
 ('Подход', 0.20892634987831116),
 ('прикладного', 0.20647269487380981),
 ('Правовое', 0.20550373196601868)]

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [8]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [9]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

Отображение пространства векторов:

In [10]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression().fit(X_train, Y_train)

print(mapping.score(X_train, Y_train))
print(mapping.coef_)

0.5865652841756943
[[-0.07448167  0.02855696  0.00477273 ...  0.05515346  0.07882929
   0.02582783]
 [-0.02114181  0.05680183  0.03354495 ...  0.04378411  0.01170791
   0.01858507]
 [ 0.14141525 -0.02344767  0.04475048 ... -0.00047052  0.04203083
  -0.03964063]
 ...
 [ 0.02423565  0.01803995 -0.04328831 ... -0.05709656  0.07869115
  -0.01255315]
 [ 0.00024532  0.01675793 -0.00442129 ...  0.05399173  0.07159225
  -0.0586846 ]
 [-0.04930586 -0.02664168 -0.03720228 ...  0.03776162  0.02120709
  -0.03378878]]


Посмотрим на соседей вектора слова "серпень" ("август" на русском) после линейного преобразования:

In [11]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8620682954788208),
 ('июнь', 0.8531144261360168),
 ('март', 0.8480008244514465),
 ('сентябрь', 0.8477311730384827),
 ('октябрь', 0.8431620597839355),
 ('февраль', 0.8424289226531982),
 ('ноябрь', 0.8402664065361023),
 ('июль', 0.83543860912323),
 ('август', 0.8232007026672363),
 ('декабрь', 0.8181025385856628)]

In [12]:
ru_emb.most_similar(mapping.predict(uk_emb["кіт"].reshape(1, -1)))

[('кот', 0.7645193338394165),
 ('котик', 0.6762055158615112),
 ('щенок', 0.6658129096031189),
 ('пес', 0.6568679809570312),
 ('котенок', 0.6546679735183716),
 ('кошка', 0.6465265154838562),
 ('кролик', 0.6448252201080322),
 ('пёс', 0.6283379793167114),
 ('хомяк', 0.6185210943222046),
 ('заяц', 0.6175106763839722)]

Мы можем видеть, что окрестность этого векторного представления состоит из разных месяцев.

В качестве показателя качества мы будем использовать точность top-1, top-5 и top-10 (для каждого преобразованного украинского векторного представления мы подсчитываем, сколько правильных пар найдено в первых N ближайших соседях в русском пространстве векторных представлений).

In [13]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""

def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs): 
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in ru_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [14]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [15]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [16]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [17]:
print(precision_top1)
print(precision_top5)

0.6744186046511628
0.8255813953488372


Попробуем улучшить полученный результат:

In [18]:
""" 
:returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
"""

def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [19]:
W = learn_transform(X_train, Y_train)

print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6831395348837209
0.8372093023255814


Получили чуть большую точность.

Теперь мы готовы создать простой переводчик с украинского на русский - для каждого слова в исходном языке мы находим ближайшее в векторном пространстве для целевого языка.

In [20]:
"""
:args:
    sentence - sentence in Ukrainian (str)
:returns:
    translation - sentence in Russian (str)

* find ukrainian embedding for each word in sentence
* transform ukrainian embedding vector
* find nearest russian word and replace
"""

def translate(sentence):
    # YOUR CODE HERE
    words = sentence.split(" ")
    translation = []
    for word in words:
        try:
            emb = uk_emb[word]
            translation.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        except:
            translation.append(word)
    return " ".join(translation)

In [21]:
print(translate("яка гарна дівка"))

она красивая девка


In [22]:
print(translate("кіт зловив мишу"))

кот поймал мышку


In [23]:
print(translate("добре сало з горілкою , особливо з перцем"))

хорошо сало со водкой , особенно со перцем


In [24]:
with open("simple_text_sample.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]    

In [ ]:
uk_sentences

In [25]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: ﻿їхав пан з кучером іваном у далеку дорогу.
dst: ﻿їхав господин со кучером іваном во даль дорогу.

src: вони їхали мовчки, але мовчанка обридла.
dst: они ехали мовчки, конечно мовчанка обридла.

src: пан і надумав поговорити.
dst: господин и надумав поговорити.

src: в цей час вискочив заєць.
dst: во этот момент выскочил заєць.

src: пан і почав свою розмову про зайця.
dst: господин и начал свою разговор о зайця.

src: — от у мене в лісі водяться зайці, тільки не такі, як оце пострибав маленький, а великі.
dst: — из во меня во лесу водятся зайці, только не такі, как вот пострибав маленький, а великі.

src: я з-за границі привіз на розплід.
dst: мной из-за границы привез по розплід.

src: одного разу я зібрався на полювання, взяв із собою чоловіка і десять загоничів.
dst: одного раз мной собрался по полювання, взял со собой мужа и десять загоничів.

src: вони нагнали на мене зайців, а я їх тільки — бах! та бах!
dst: они нагнали по меня зайців, а мной их только — бах! и бах!

src: т

Получиен весьма несовершенный перевод с пропуском многих слов, так произошло из-за ограничения размеров векторных представлений (больший объём представлений приводит к ошибкам в работе функций связанных с недостатком памяти).

## Часть 2: Переводчик для французского и португальского языков

Загружем векторные представления:

In [2]:
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec", binary = False, limit = 100000)
pt_emb = KeyedVectors.load_word2vec_format("cc.pt.300.vec", binary = False, limit = 100000)

### Перевод с французского на португальский:

In [3]:
def load_word_pairs(filename):
    fr_pt_pairs = []
    fr_vectors = []
    pt_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            fr, pt = line.rstrip().split("\t")
            if fr not in fr_emb or pt not in pt_emb:
                continue
            fr_pt_pairs.append((fr, pt))
            fr_vectors.append(fr_emb[fr])
            pt_vectors.append(pt_emb[pt])
            
    return fr_pt_pairs, np.array(fr_vectors), np.array(pt_vectors)

In [4]:
fr_pt_train, X_train, Y_train = load_word_pairs("fr-pt.train.txt")
fr_pt_test, X_test, Y_test = load_word_pairs("fr-pt.test.txt")

Получим две матрицы преобразования M1 и M2 для первых 300 пар и следующих 300 пар соответственно:

In [5]:
""" 
:returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
"""

def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

In [6]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Вычислим точность для M1 и M2 на разных наборах пар:

In [7]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""
    
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x, _) in pt_emb.most_similar([mapped_vector], topn = topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [8]:
print("pair1 M1: {}".format(precision(fr_pt_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(fr_pt_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(fr_pt_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(fr_pt_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.7166666666666667
pair1 M2: 0.7166666666666667
pair2 M1: 0.7541528239202658
pair2 M2: 0.7541528239202658


Для пары французский - португальский языки получили точность около 73%.

Вычислим норму фробениуса для матрицы $|M1-M2|$:

In [9]:
M1_M2 = np.linalg.norm((M1-M2), ord ='fro')
M1_M2

0.0

Вычислим отношение фробениусовых норм $\frac{|M1-M2|}{|M1|}$:

In [10]:
M1_M2/np.linalg.norm(M1, ord ='fro')

0.0

Проверим ортогональность матриц:

In [11]:
print(np.matmul(M1, M1.T))

[[ 1.0000001e+00  7.4505806e-09 -1.8626451e-09 ...  7.4505806e-09
  -3.5390258e-08  3.7252903e-09]
 [ 7.4505806e-09  9.9999994e-01 -2.3283064e-08 ...  2.3283064e-09
   2.2351742e-08  2.2351742e-08]
 [-1.8626451e-09 -2.3283064e-08  9.9999994e-01 ...  1.8160790e-08
  -1.3969839e-09  0.0000000e+00]
 ...
 [ 7.4505806e-09  2.3283064e-09  1.8160790e-08 ...  1.0000000e+00
   1.8626451e-09  1.5832484e-08]
 [-3.5390258e-08  2.2351742e-08 -1.3969839e-09 ...  1.8626451e-09
   1.0000000e+00 -1.4901161e-08]
 [ 3.7252903e-09  2.2351742e-08  0.0000000e+00 ...  1.5832484e-08
  -1.4901161e-08  9.9999994e-01]]


M1 ортогональна.

In [12]:
print(np.matmul(M2, M2.T))

[[ 1.0000001e+00  7.4505806e-09 -1.8626451e-09 ...  7.4505806e-09
  -3.5390258e-08  3.7252903e-09]
 [ 7.4505806e-09  9.9999994e-01 -2.3283064e-08 ...  2.3283064e-09
   2.2351742e-08  2.2351742e-08]
 [-1.8626451e-09 -2.3283064e-08  9.9999994e-01 ...  1.8160790e-08
  -1.3969839e-09  0.0000000e+00]
 ...
 [ 7.4505806e-09  2.3283064e-09  1.8160790e-08 ...  1.0000000e+00
   1.8626451e-09  1.5832484e-08]
 [-3.5390258e-08  2.2351742e-08 -1.3969839e-09 ...  1.8626451e-09
   1.0000000e+00 -1.4901161e-08]
 [ 3.7252903e-09  2.2351742e-08  0.0000000e+00 ...  1.5832484e-08
  -1.4901161e-08  9.9999994e-01]]


M2 ортогональна.

### Перевод с португальского на французский:

In [27]:
def load_word_pairs(filename):
    pt_fr_pairs = []
    fr_vectors = []
    pt_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            pt, fr = line.rstrip().split("\t")
            if pt not in pt_emb or fr not in fr_emb:
                continue
            pt_fr_pairs.append((pt, fr))
            fr_vectors.append(fr_emb[fr])
            pt_vectors.append(pt_emb[pt])
            
    return pt_fr_pairs, np.array(pt_vectors), np.array(fr_vectors)

In [28]:
pt_fr_train, X_train, Y_train = load_word_pairs("pt-fr.train.txt")
pt_fr_test, X_test, Y_test = load_word_pairs("pt-fr.test.txt")

Получим две матрицы преобразования M1 и M2 для первых 300 пар и следующих 300 пар соответственно:

In [29]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

Вычислим точность для M1 и M2 на разных наборах пар:

In [30]:
"""
:args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
:returns:
    precision_val, float number, total number of words for those we can find right translation at top K.
"""

def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x, _) in fr_emb.most_similar([mapped_vector], topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [31]:
print("pair1 M1: {}".format(precision(pt_fr_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(pt_fr_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(pt_fr_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(pt_fr_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.63
pair1 M2: 0.63
pair2 M1: 0.6744186046511628
pair2 M2: 0.6744186046511628


Для пары португальский - французский языки получили точность около 65%. Это меньше, чем для пары французский - португальский.

Вычислим норму фробениуса для матрицы  $|M1-M2|$:

In [32]:
M1_M2 = np.linalg.norm((M1-M2), ord = "fro")
M1_M2

0.0

Вычислим отношение фробениусовых норм $\frac{|M1-M2|}{|M1|}$:

In [33]:
M1_M2/np.linalg.norm(M1, ord = "fro")

0.0

Проверим ортогональность матриц:

In [34]:
print(np.matmul(M1, M1.T))

[[ 1.0000001e+00  2.4680048e-08  3.1664968e-08 ... -3.1432137e-09
   6.5192580e-09 -2.6077032e-08]
 [ 2.4680048e-08  1.0000000e+00 -3.7252903e-09 ...  3.7252903e-09
   2.2351742e-08  1.0535587e-08]
 [ 3.1664968e-08 -3.7252903e-09  9.9999994e-01 ... -1.9557774e-08
  -1.6763806e-08 -9.3132257e-10]
 ...
 [-3.1432137e-09  3.7252903e-09 -1.9557774e-08 ...  1.0000000e+00
  -2.2817403e-08 -2.7939677e-09]
 [ 6.5192580e-09  2.2351742e-08 -1.6763806e-08 ... -2.2817403e-08
   9.9999994e-01  1.4901161e-08]
 [-2.6077032e-08  1.0535587e-08 -9.3132257e-10 ... -2.7939677e-09
   1.4901161e-08  9.9999988e-01]]


M1 ортогональна.

In [35]:
print(np.matmul(M2, M2.T))

[[ 1.0000001e+00  2.4680048e-08  3.1664968e-08 ... -3.1432137e-09
   6.5192580e-09 -2.6077032e-08]
 [ 2.4680048e-08  1.0000000e+00 -3.7252903e-09 ...  3.7252903e-09
   2.2351742e-08  1.0535587e-08]
 [ 3.1664968e-08 -3.7252903e-09  9.9999994e-01 ... -1.9557774e-08
  -1.6763806e-08 -9.3132257e-10]
 ...
 [-3.1432137e-09  3.7252903e-09 -1.9557774e-08 ...  1.0000000e+00
  -2.2817403e-08 -2.7939677e-09]
 [ 6.5192580e-09  2.2351742e-08 -1.6763806e-08 ... -2.2817403e-08
   9.9999994e-01  1.4901161e-08]
 [-2.6077032e-08  1.0535587e-08 -9.3132257e-10 ... -2.7939677e-09
   1.4901161e-08  9.9999988e-01]]


M2 ортогональна.